In [1]:
%reload_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [3]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

**TABLE**

In [6]:
%%sql
CREATE TABLE User (
    userID INTEGER PRIMARY KEY AUTOINCREMENT,
    preference CHAR(50),
    email CHAR(50) NOT NULL UNIQUE CHECK(email LIKE '%@%.%')
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS Item (
    ItemID INTEGER PRIMARY KEY AUTOINCREMENT,
    title CHAR(50) NOT NULL,
    type CHAR(50) NOT NULL,
    author CHAR(50),
    status CHAR(50)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [10]:
%%sql
CREATE TABLE IF NOT EXISTS CD (
    ItemID INTEGER PRIMARY KEY,
    duration INTEGER NOT NULL,
    FOREIGN KEY (ItemID) REFERENCES Item (ItemID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [12]:
%%sql
CREATE TABLE IF NOT EXISTS OnlineItem (
    ItemID INTEGER PRIMARY KEY,
    url CHAR(50) NOT NULL,
    FOREIGN KEY (ItemID) REFERENCES Item (ItemID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [14]:
%%sql
CREATE TABLE IF NOT EXISTS Loan (
    LoanID INTEGER PRIMARY KEY AUTOINCREMENT,
    due DATE NOT NULL,
    userID INTEGER NOT NULL,
    ItemID INTEGER NOT NULL,
    FOREIGN KEY (userID) REFERENCES User (userID) ON DELETE CASCADE,
    FOREIGN KEY (ItemID) REFERENCES Item (ItemID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [16]:
%%sql
CREATE TABLE IF NOT EXISTS Borrows (
    userID INTEGER NOT NULL,
    LoanID INTEGER NOT NULL,
    PRIMARY KEY (userID, LoanID),
    FOREIGN KEY (userID) REFERENCES User (userID) ON DELETE CASCADE,
    FOREIGN KEY (LoanID) REFERENCES Loan (LoanID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [18]:
%%sql 
CREATE TABLE IF NOT EXISTS Fine (
    LoanID INTEGER PRIMARY KEY,
    amount DECIMAL(10,2) NOT NULL CHECK(amount >= 0),
    status CHAR(50) CHECK(status IN ('Unpaid', 'Paid')),
    FOREIGN KEY (LoanID) REFERENCES Loan (LoanID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [20]:
%%sql
CREATE TABLE IF NOT EXISTS Event (
    EventID INTEGER PRIMARY KEY AUTOINCREMENT,
    type CHAR(50) NOT NULL,
    roomID INTEGER,
    FOREIGN KEY (roomID) REFERENCES Room(roomID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [22]:
%%sql
CREATE TABLE IF NOT EXISTS Attends (
    userID INTEGER NOT NULL,
    EventID INTEGER NOT NULL,
    PRIMARY KEY (userID, EventID),
    FOREIGN KEY (userID) REFERENCES User (userID) ON DELETE CASCADE,
    FOREIGN KEY (EventID) REFERENCES Event (EventID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [24]:
%%sql
CREATE TABLE IF NOT EXISTS Room (
    roomID INTEGER PRIMARY KEY AUTOINCREMENT,
    capacity INTEGER NOT NULL
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [26]:
%%sql
CREATE TABLE IF NOT EXISTS Uses (
    EventID INTEGER NOT NULL,
    roomID INTEGER NOT NULL,
    PRIMARY KEY (EventID, roomID),
    FOREIGN KEY (EventID) REFERENCES Event (EventID) ON DELETE CASCADE,
    FOREIGN KEY (roomID) REFERENCES Room (roomID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [28]:
%%sql
CREATE TABLE IF NOT EXISTS Personnel (
    staffID INTEGER PRIMARY KEY AUTOINCREMENT,
    name CHAR(50) NOT NULL,
    position CHAR(50) NOT NULL,
    salary DECIMAL(10,2) NOT NULL CHECK (salary > 0)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [30]:
%%sql
CREATE TABLE IF NOT EXISTS Manages (
    staffID INTEGER NOT NULL,
    EventID INTEGER NOT NULL,
    PRIMARY KEY (staffID, EventID),
    FOREIGN KEY (staffID) REFERENCES Personnel (staffID) ON DELETE CASCADE,
    FOREIGN KEY (EventID) REFERENCES Event (EventID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [32]:
%%sql
CREATE TABLE IF NOT EXISTS NotificationQueue (
    notificationID INTEGER PRIMARY KEY AUTOINCREMENT,
    userID INTEGER NOT NULL,
    eventID INTEGER NOT NULL,
    email CHAR(50) NOT NULL,
    message CHAR(50) NOT NULL,
    status CHAR(50) DEFAULT 'Pending' CHECK(status IN ('Pending', 'Sent')),
    FOREIGN KEY (userID) REFERENCES User(userID),
    FOREIGN KEY (eventID) REFERENCES Event(eventID)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [34]:
%%sql
CREATE TABLE IF NOT EXISTS HelpRequests (
    requestID INTEGER PRIMARY KEY AUTOINCREMENT,
    email CHAR(50) NOT NULL,
    message CHAR(50) NOT NULL
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [36]:
%%sql 
SELECT name FROM sqlite_master WHERE type='table';


Running query in 'sqlite:///library.db'

name
User
sqlite_sequence
Item
CD
OnlineItem
Loan
Borrows
Fine
Event
Attends


**VALUE**

In [39]:
%%sql
INSERT INTO User (userID, email, preference)
VALUES
    (1, 'alice.brown@example.com', 'Book Club'),
    (2, 'bob.smith@example.com', 'Book Club'),
    (3, 'carol.jones@example.com', 'Film Screening'),
    (4, 'dave.wilson@example.com', 'Art Show'),
    (5, 'eve.davis@example.com', 'Book Club'),
    (6, 'frank.miller@example.com', 'Art Show'),
    (7, 'grace.lee@example.com', 'Film Screening'),
    (8, 'henry.taylor@example.com', 'Book Club'),
    (9, 'ivy.anderson@example.com', 'Art Show'),
    (10, 'jack.thomas@example.com', 'Book Club');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [41]:
%%sql
INSERT INTO Item (ItemID, Title, Type, Author, Status)
VALUES
    (101, 'The Great Gatsby', 'Book', 'F. Scott Fitzgerald', 'Available'),
    (102, '1984', 'Book', 'George Orwell', 'Borrowed'),
    (103, 'National Geographic', 'Magazine', 'Various', 'Available'),
    (104, 'Time', 'Magazine', 'Various', 'Reserved'),
    (105, 'Thriller', 'CD', 'Michael Jackson', 'Available'),
    (106, 'Inception', 'DVD', 'Christopher Nolan', 'Available'),
    (107, 'Learn Python', 'Online Book', 'John Doe', 'Available'),
    (108, 'Interstellar Soundtrack', 'CD', 'Hans Zimmer', 'Borrowed'),
    (109, 'The New Yorker', 'Magazine', 'Various', 'Available'),
    (110, 'Science Journal', 'Scientific Journal', 'Various', 'Available');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [43]:
%%sql
INSERT INTO Loan (LoanID, UserID, ItemID, due)
VALUES
    (201, 1, 102, '2025-04-15'),
    (202, 2, 105, '2025-04-20'),
    (203, 3, 108, '2025-04-18'),
    (204, 4, 101, '2025-04-12'),
    (205, 5, 107, '2025-04-22'),
    (206, 6, 106, '2025-04-16'),
    (207, 7, 104, '2025-04-19'),
    (208, 8, 103, '2025-04-17'),
    (209, 9, 110, '2025-04-21'),
    (210, 10, 109, '2025-04-25');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [45]:
%%sql
INSERT INTO Fine (LoanID, Amount, Status)
VALUES
    (201, 10.00, 'Unpaid'),
    (202, 0.00, 'Paid'),
    (203, 5.00, 'Unpaid'),
    (204, 0.00, 'Paid'),
    (205, 0.00, 'Paid'),
    (206, 0.00, 'Paid'),
    (207, 3.00, 'Unpaid'),
    (208, 0.00, 'Paid'),
    (209, 8.00, 'Unpaid'),
    (210, 0.00, 'Paid');

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [47]:
%%sql
INSERT INTO Event (EventID, Type, RoomID)
VALUES
    (401, 'Book Club', 1),
    (402, 'Film Screening', 2),
    (403, 'Art Show', 3),
    (404, 'Tech Talk', 4),
    (405, 'Cooking Class', 5),
    (406, 'History Lecture', 2),
    (407, 'Science Seminar', 3),
    (408, 'Music Concert', 1),
    (409, 'Poetry Reading', 4),
    (410, 'Children Storytime', 5);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [49]:
%%sql
INSERT INTO Room (RoomID, Capacity)
VALUES
    (501, 30),
    (502, 50),
    (503, 40),
    (504, 25),
    (505, 20),
    (506, 60),
    (507, 15),
    (508, 35),
    (509, 45),
    (510, 55);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [51]:
%%sql
INSERT INTO Personnel (staffID, name, position, salary)
VALUES
    (601, 'Alice Zhang', 'Librarian', 52000.00),
    (602, 'Bob Lee', 'Event Manager', 58000.00),
    (603, 'Carol Wang', 'Event Manager', 50000.00),
    (604, 'David Kim', 'Technician', 47000.00),
    (605, 'Eve Liu', 'Assistant Librarian', 42000.00),
    (606, 'Frank Chen', 'IT Support', 54000.00),
    (607, 'Grace Huang', 'Head Librarian', 68000.00),
    (608, 'Henry Zhao', 'Security Officer', 40000.00),
    (609, 'Ivy Xu', 'Cataloging Specialist', 49000.00),
    (610, 'Jack Ma', 'Volunteer Coordinator', 45000.00);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [53]:
%%sql
INSERT INTO Manages (staffID, EventID)
VALUES
    (602, 401),
    (603, 402),
    (602, 403),
    (603, 404),
    (602, 405),
    (603, 406),
    (602, 407),
    (603, 408),
    (602, 409),
    (603, 410);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [55]:
%%sql
INSERT INTO Uses (EventID, roomID)
VALUES
    (401, 501),
    (402, 502),
    (403, 503),
    (404, 504),
    (405, 505),
    (406, 506),
    (407, 507),
    (408, 508),
    (409, 509),
    (410, 510);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [57]:
%%sql
INSERT INTO Attends (userID, EventID)
VALUES
    (1, 401),
    (2, 401),
    (3, 402),
    (4, 403),
    (5, 401),
    (6, 403),
    (7, 402),
    (8, 401),
    (9, 403),
    (10, 401);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [59]:
%%sql
INSERT INTO Borrows (userID, LoanID)
VALUES
    (1, 201),
    (2, 202),
    (3, 203),
    (4, 204),
    (5, 205),
    (6, 206),
    (7, 207),
    (8, 208),
    (9, 209),
    (10, 210);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [61]:
%%sql
INSERT INTO NotificationQueue (notificationID, userID, eventID, email, message)
VALUES
    (701, 1, 401, 'alice.brown@example.com', 'You may be interested in this event. Would you like to come?'),
    (702, 2, 401, 'bob.smith@example.com', 'You may be interested in this event. Would you like to come?'),
    (703, 3, 402, 'carol.jones@example.com', 'You may be interested in this event. Would you like to come?'),
    (704, 4, 403, 'dave.wilson@example.com', 'You may be interested in this event. Would you like to come?'),
    (705, 5, 401, 'eve.davis@example.com', 'You may be interested in this event. Would you like to come?'),
    (706, 6, 403, 'frank.miller@example.com', 'You may be interested in this event. Would you like to come?'),
    (707, 7, 402, 'grace.lee@example.com', 'You may be interested in this event. Would you like to come?'),
    (708, 8, 401, 'henry.taylor@example.com', 'You may be interested in this event. Would you like to come?'),
    (709, 9, 403, 'ivy.anderson@example.com', 'You may be interested in this event. Would you like to come?'),
    (710, 10, 401, 'jack.thomas@example.com', 'You may be interested in this event. Would you like to come?');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [63]:
%%sql
INSERT INTO HelpRequests (requestID, email, message)
VALUES
    (801, 'alice.brown@example.com', 'We are currently processing your request.'),
    (802, 'bob.smith@example.com', 'We are currently processing your request.'),
    (803, 'carol.jones@example.com', 'We are currently processing your request.'),
    (804, 'dave.wilson@example.com', 'We are currently processing your request.'),
    (805, 'eve.davis@example.com', 'We are currently processing your request.'),
    (806, 'frank.miller@example.com', 'We are currently processing your request.'),
    (807, 'grace.lee@example.com', 'We are currently processing your request.'),
    (808, 'henry.taylor@example.com', 'We are currently processing your request.'),
    (809, 'ivy.anderson@example.com', 'We are currently processing your request.'),
    (810, 'jack.thomas@example.com', 'We are currently processing your request.');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

**TRIGGER**

In [66]:
%%sql
CREATE TRIGGER update_item_status_on_borrow
AFTER INSERT ON Loan
FOR EACH ROW
BEGIN
    UPDATE Item
    SET status = 'borrowed'
    WHERE ItemID = NEW.ItemID;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [68]:
%%sql
CREATE TRIGGER update_item_status_on_return
AFTER DELETE ON Loan
FOR EACH ROW
BEGIN
    UPDATE Item
    SET status = 'available'
    WHERE ItemID = OLD.ItemID;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [70]:
%%sql
CREATE TRIGGER IF NOT EXISTS trigger_notify_users_on_new_event
AFTER INSERT ON Event
FOR EACH ROW
BEGIN
    INSERT INTO NotificationQueue (userID, eventID, email, message)
    SELECT 
        userID,
        NEW.EventID,
        email,
        'Hi! Here is a new event you might like has been posted: ' || NEW.type
    FROM User
    WHERE preference = NEW.type;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [72]:
%%sql
CREATE TRIGGER prevent_borrow_if_unavailable
BEFORE INSERT ON Loan
FOR EACH ROW
BEGIN
    SELECT 
    CASE
        WHEN (SELECT status FROM Item WHERE ItemID = NEW.ItemID) = 'borrowed'
        THEN RAISE(ABORT, 'This item is already borrowed.')
    END;
END;


Running query in 'sqlite:///library.db'

++
||
++
++